In [1]:
import geopandas as gpd

In [3]:
import os

In [2]:
path = r'T:\Trans Projects\Model Development\UrbanSim_LandUse\Output\Simulation_47_Final_RTP'

In [4]:
shp = 'parcel_fabric.shp'

In [5]:
parcel_data = gpd.read_file(os.path.join(path, shp))

In [6]:
parcel_data.columns

Index(['btype', 'nrsqft', 'rsqft', 'du', 'yrbuilt', 'lpid', 'pundev',
       'dev_land', 'developed', 'obtype', 'orsqft', 'onrsqft', 'odu', 'zid',
       'rezoned', 'city', 'annexed', 'ozid', 'geometry'],
      dtype='object')

In [7]:
parcel_data.head()

,btype,nrsqft,rsqft,du,yrbuilt,lpid,pundev,dev_land,developed,obtype,orsqft,onrsqft,odu,zid,rezoned,city,annexed,ozid,geometry
0,(),0.0,0.0,0.0,0.0,24284,0.00,1.123953e+04,0,(),0.0,0.0,0.0,119,0,None,0,119,"MULTIPOLYGON (((4254437.869 843733.606, 425444..."
1,"(5100,)",1662.0,0.0,0.0,1969.0,24356,0.15,1.973955e+06,0,"(5100,)",0.0,1662.0,0.0,119,0,None,0,119,"POLYGON ((4254479.021 845048.443, 4254437.869 ..."
2,"(1110,)",0.0,3468.0,1.0,2006.0,24391,0.00,1.099604e+05,0,"(1110,)",3468.0,0.0,1.0,62,0,None,0,62,"POLYGON ((4250857.967 845652.695, 4250847.053 ..."
3,"(1110,)",0.0,1296.0,1.0,1984.0,24408,0.00,6.144933e+04,0,"(1110,)",1296.0,0.0,1.0,62,0,None,0,62,"POLYGON ((4250865.647 845846.211, 4250859.554 ..."
4,"(1110,)",0.0,2528.0,1.0,2015.0,24419,0.00,6.097905e+04,0,"(1110,)",2528.0,0.0,1.0,62,0,None,0,62,"POLYGON ((4250871.702 845998.498, 4250865.647 ..."


In [9]:
parcel_data.shape

(78216, 19)

In [11]:
sorted(parcel_data['yrbuilt'].unique())

[0.0,
 1800.0,
 1850.0,
 1853.0,
 1860.0,
 1863.0,
 1870.0,
 1880.0,
 1885.0,
 1887.0,
 1888.0,
 1889.0,
 1890.0,
 1891.0,
 1892.0,
 1893.0,
 1894.0,
 1895.0,
 1896.0,
 1898.0,
 1899.0,
 1899.5,
 1900.0,
 1901.0,
 1901.5,
 1902.0,
 1903.0,
 1904.0,
 1905.0,
 1906.0,
 1907.0,
 1908.0,
 1909.0,
 1910.0,
 1911.0,
 1912.0,
 1913.0,
 1914.0,
 1915.0,
 1916.0,
 1917.0,
 1917.5,
 1918.0,
 1919.0,
 1919.5,
 1920.0,
 1920.5,
 1921.0,
 1922.0,
 1923.0,
 1923.5,
 1924.0,
 1924.5,
 1925.0,
 1926.0,
 1926.5,
 1927.0,
 1927.5,
 1928.0,
 1928.5,
 1929.0,
 1930.0,
 1931.0,
 1931.5,
 1932.0,
 1932.5,
 1933.0,
 1933.5,
 1934.0,
 1934.5,
 1935.0,
 1935.5,
 1936.0,
 1936.5,
 1937.0,
 1937.5,
 1938.0,
 1938.5,
 1939.0,
 1939.5,
 1940.0,
 1940.5,
 1941.0,
 1941.5,
 1942.0,
 1942.5,
 1943.0,
 1943.5,
 1944.0,
 1944.5,
 1945.0,
 1945.5,
 1946.0,
 1946.5,
 1947.0,
 1947.5,
 1948.0,
 1948.5,
 1949.0,
 1949.5,
 1950.0,
 1950.5,
 1951.0,
 1951.5,
 1952.0,
 1952.5,
 1953.0,
 1953.5,
 1954.0,
 1954.5,
 1955.0,
 195

In [13]:
parcel_data[parcel_data['yrbuilt'] > 2020]['odu'].describe()

count    2703.000000
mean        0.912320
std        10.638641
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max       536.000000
Name: odu, dtype: float64

In [15]:
parcel_data[parcel_data['yrbuilt'] < 2020]['onrsqft'].describe()

count    7.551300e+04
mean     9.496663e+02
std      1.209315e+04
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.117553e+06
Name: onrsqft, dtype: float64

In [16]:
parcel_data[parcel_data['yrbuilt'] > 2020]['nrsqft'].describe()

count    2.703000e+03
mean     1.130654e+04
std      4.718592e+04
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      9.240500e+03
max      1.038311e+06
Name: nrsqft, dtype: float64

In [22]:
parcel_data['btype'].dtypes

dtype('O')

In [30]:
parcel_data[['btype']].head()

,btype
0,()
1,"(5100,)"
2,"(1110,)"
3,"(1110,)"
4,"(1110,)"


In [33]:
parcel_data.btype.apply(len).max()

36

In [39]:
parcel_data['btype_len'] = parcel_data.btype.apply(len)
parcel_data.loc[parcel_data.btype_len == parcel_data.btype_len.max(), 'btype']

24842    (1110, 4100, 5100, 6120, 6500, 7100)
Name: btype, dtype: object

In [40]:
parcel_data['btype1'] = parcel_data['btype'].apply(lambda x: str(x)[1:-2].split(', ')[0])

0            
1        5100
2        1110
3        1110
4        1110
         ... 
78211    1110
78212    1110
78213    4100
78214    1110
78215    1110
Name: btype, Length: 78216, dtype: object